In [1]:
!pip install accelerate
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from torch.utils.data import Dataset
import accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
data = pd.read_csv('Finance_data.csv')

In [3]:
data.dropna(subset=['What are your savings objectives?', 'Avenue'], inplace=True)

In [4]:
data['input_text'] = data['What are your savings objectives?'].str.lower().str.strip()
data['output_text'] = data['Avenue'].str.lower().str.strip()

In [5]:
data = data[data['input_text'].str.strip() != '']
data = data[data['output_text'].str.strip() != '']
data.drop_duplicates(subset=['input_text', 'output_text'], inplace=True)

In [6]:
tokenizer = AutoTokenizer.from_pretrained('tiiuae/falcon-7b')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [7]:
tokenizer.pad_token = tokenizer.eos_token

In [8]:
def tokenize_data(data):
    return tokenizer(
        list(data['input_text']),
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        max_length=128  # Set maximum length for tokens
    ), tokenizer(
        list(data['output_text']),
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        max_length=128
    )

In [9]:
train_encodings, train_labels = tokenize_data(data)

In [10]:
class FinancialAdviceDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Return input and label tensors
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx]
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

In [11]:
train_dataset = FinancialAdviceDataset(train_encodings, train_labels)

In [12]:
import torch

In [13]:
torch.backends.cudnn.benchmark = True

In [15]:
torch.backends.cudnn.deterministic = False
device_map = {"": 0}

In [16]:
model = AutoModelForCausalLM.from_pretrained('tiiuae/falcon-7b',torch_dtype=torch.float16, device_map=device_map)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]